Quantum noise can be characterized into coherent and incoherent sources oferrors that arise during a computation. Coherent noise is commonly due to systematic errors originationg from device miscalibrations, for example, gates implementing a rotation $\theta+\epsilon$ instead of $\theta$  

Incoherent noise has its origins in quantum states being entangled wit the environment due to decoherence. This leads to mixed states which are probability distributions over pure states and are described by employing the density matrix formalism.

We can model incoherent noise via quantum channels which are liniear, completely positive, and trace preserving maps. These maps are called Kraus Operators, $\{K_i\}$, chich satisfy the condition

$$ \sum_i K_i^\dagger K_i = I $$


The bit-flip channel flips the qubit with probability *p* and leaves it unchanged with probability 1-p. This can be represented by employting Kraus operators:
$$K_0 = \sqrt{1-p} \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} $$
$$K_0 = \sqrt{p} \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} $$

Let's implement the bit-lfip channel using CUDA-Q:

In [2]:
import cudaq
from cudaq import spin

import numpy as np

cudaq.set_target('density-matrix-cpu') # To model quantum noise, we nned to utilize the density matrix simulator target

qubit_counts = 2

@cudaq.kernel
def kernel(qubit_count: int):
    qvector = cudaq.qvector(qubit_count)
    x(qvector)



print(cudaq.draw(kernel, qubit_counts))

     ╭───╮
q0 : ┤ x ├
     ├───┤
q1 : ┤ x ├
     ╰───╯



In [3]:
# In the ideal noiseless case, we get |11> 100% of the time

ideal_counts = cudaq.sample(kernel, qubit_counts, shots_count = 100)
ideal_counts.dump()

{ 11:100 }


In [ ]:
# First, we weil define an out of the box noise channel. In this case, we choose deploarization noise. This deploarization will result in 
# the qubit state decaying into a mix of ths basis states, |0> and |1>, with our provided probability.
error_prob = np.linspace(0.05, 0.29, 0.2)
depolarizing_channel = cudaq.DepolarizationChannel(error_prob)

# Other built in noise model
bit_flip = cudaq.BitFlipChannel(error_prob)
phase_flip = cudaq.PhaseFlipChannel(error_prob)
amplitude_damping = cudaq.AmplitudeDampingChannel(error_prob)

# We can also define our own, custom noise channels using Kraus
kraus_0 = np.sqrt(1 - error_prob) * np.array([[1.0, 0.0],
                                                     [0.0, 1.0]],
                                                    dtype=np.complex128)

kraus_1 = np.sqrt(error_prob) * np.array([[0.0, 1.0],
                                                 [1.0, 0.0]],
                                                dtype=np.complex128)

# Add the Kraus Operator to create a quantum channel.
bitflip_channel = cudaq.KrausChannel([kraus_0, kraus_1])

In [ ]:
# Add noise channels to our noise model. 
noise_model = cudaq.NoiseModel()

# Apply channel to any X-gate on the 0th qubit
noise_model.add_channel("x", [0], depolarizing_channel)
# Apply channel to any X-gate on the 1st qubit
noise_model.add_channel('x', [1]
                        , depolarizing_channel)
noise_model.add_channel('mz', [1], bitflip_channel) # measure할 때 bitflip 채널 적용


# Due to the impact of noise, our measurement results will vary from the ideal case.
noisy_counts = cudaq.sample(kernel, qubit_counts, noise_model = noise_model, shots_count = 1000)

noisy_counts.dump()

{ 10:61 11:939 }


In [15]:
# We can also use noise noedls with the observe function
hamiltonian = spin.z(0)

noisy_result = cudaq.observe(kernel, hamiltonian, qubit_counts, noise_model = noise_model)
noisy_result.expectation()

-0.8666666666666665

In addition to gate-based noise injection, CUDA-Q also supports fine-grained, explict noiseinjection via the ```cudaq.apply_noise``` function. You can place this method in your kernels, and inject specific noise on specific qubits wherever you need

In [4]:
from cudaq import spin
import cudaq
import numpy as np
c = np.array([1, 0., 0., 0.], dtype=cudaq.complex())


noise_model = cudaq.NoiseModel()
@cudaq.kernel
def kernel():
    q = cudaq.qvector(c)
    x.ctrl(q[0], q[1])
    cudaq.apply_noise(cudaq.BitFlipChannel, 0.1, q[0]) # Inject a Y error, Y error means bit and phase flip


cudaq.sample(kernel, noise_model=noise_model).dump()

{ 00:905 10:95 }


In [ ]:
import cudaq
from cudaq import spin
import numpy as np

noise_model = cudaq.NoiseModel()

@cudaq.kernel
def circuit(prob_amp: list[np.array]):
    qc = cudaq.qvector(prob_amp)
    cudaq.apply_noise(cudaq.BitFlipChannel, 0.1, qc[0]) # Inject a Y error, Y error means bit and phase flip


prob_amp = [1/np.sqrt(3), np.sqrt(2/3)]

cudaq.observe(circuit, spin.z(0), prob_amp, noise_model=noise_model).expectation()

error: 'quake.init_state' op invalid data pointer type


RuntimeError: could not compile code for 'circuit'.

In [ ]:
@cudaq.kernel
def inject_noise():
    q = cudaq.qvector(3)

    cudaq.apply_noise(cudaq.DepolarizationChannel, 0.1, q[0])

    h(q[1])
    x.ctrl(q[1], q[2])

    cudaq.apply_noise(cudaq.YError, 0.1, q[1]) # Inject a Y error, Y error means bit and phase flip

    cudaq.apply_noise(cudaq.Pauli1, 0.1, 0.1, 0.1, q[2]) # Apply a general pauli noise channel, where 3 values indicate the probability of X, Y, and Z errors

noise = cudaq.NoiseModel()
counts = cudaq.sample(inject_noise, noise_model=noise) # 내부에 국소 임의 노이즈를 넣을 때는 noise_model = noise로 설정 함. 내가 만약 외부에서 노이즈 모델을 넣고 싶으면 noise_model = noise_model로 설정
print(counts)

{ 000:335 001:116 010:132 011:344 100:29 101:8 110:13 111:23 }

